**Thêm các thư viện cần thiết**

In [1]:
import pandas as pd
import numpy as np
import math

Tên file:
- input: "house.csv"
- output: "house_preprocessing_for_dashboard.csv"

In [2]:
INPUT = "dataset/house.csv"
OUTPUT = "dataset/house_preprocessing_for_dashboard.csv"

**Các cột cần xử lí**

In [3]:
quan = 'Quận'
loai_hinh = 'Loại hình nhà ở'
giay_to = 'Giấy tờ pháp lý'
so_tang = 'Số tầng'
so_phong_ngu = 'Số phòng ngủ'
dien_tich = 'Diện tích (m2)'
dang_nha = 'Dáng nhà'

### Đọc dữ liệu

In [4]:
df = pd.read_csv(INPUT, index_col=0)
df.head()

,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích (m2),Dài (m),Rộng (m),Giá/m2 (triệu)
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4.0,5.0,46.0,NaN,NaN,86.96
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3.0,37.0,NaN,NaN,116.22
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,10.00,4.0,65.00
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100.00
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4.0,36.0,9.00,4.0,86.11


**Loại bỏ các dòng trùng lặp**

In [5]:
df.drop_duplicates(inplace=True)
len(df[df.duplicated()])

0

**Xóa bỏ các dòng có `nan` ở cột `Quận`**

In [6]:
df.dropna(subset=[quan], inplace=True)
df[df[quan].isna()]

,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích (m2),Dài (m),Rộng (m),Giá/m2 (triệu)


**Thay thế `nan` ở các cột `Loại hình nhà ở` và `Giấy tờ pháp lý` bằng giá trị `Không có`**

In [7]:
df.fillna({loai_hinh: 'Không có', giay_to: 'Không có'}, inplace=True)
df[df[loai_hinh] == 'Không có'].head()

,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích (m2),Dài (m),Rộng (m),Giá/m2 (triệu)
3928,2020-08-03,"Đường Đặng Thai Mai, Phường Quảng An, Quận Tây...",Quận Tây Hồ,Phường Quảng An,Không có,Không có,NaN,NaN,1000.0,NaN,NaN,12.600
19888,2020-07-24,"Đường Văn Phú, Phường Phúc La, Quận Hà Đông, H...",Quận Hà Đông,Phường Phúc La,Không có,Đã có sổ,NaN,4.0,50.0,NaN,NaN,136.000
23415,2020-07-22,"Đường Võ Chí Công||770, Phường Xuân La, Quận T...",Quận Tây Hồ,Phường Xuân La,Không có,Đã có sổ,NaN,NaN,33.0,9.0,3.6,93.940
26557,2020-07-20,"Đường Thái Hà, Phường Láng Hạ, Quận Đống Đa, H...",Quận Đống Đa,Phường Láng Hạ,Không có,Đã có sổ,NaN,NaN,71.0,NaN,NaN,185.920
26908,2020-07-20,"Flc Đại Mỗ, Phường Đại Mỗ, Quận Nam Từ Liêm, H...",Quận Nam Từ Liêm,Phường Đại Mỗ,Không có,Đang chờ sổ,NaN,1.0,45.0,NaN,NaN,488.888


**Cột Số tầng, Số phòng ngủ, Diện tích (m2), thực hiện bining method (xác định bin size bằng Sturge’s Rule).**

In [8]:
# Xác định biên của các giỏ
def equifreq(arr1, m):    
    a = len(arr1)
    n = int(a / m)
    borders = []
    for i in range(0, m):
        arr = []
        for j in range(i * n, (i + 1) * n):
            if j >= a:
                break
            arr = arr + [arr1[j]]
        #print(arr)
        #print(arr[0])
        borders.append(round(arr[0] - 1))
    return borders

# Tạo cột mới ứng với giá trị của cột và biên của giỏ
def apply_binning(col_name):
    col_val = df[col_name].unique() 
    col_val = col_val[~np.isnan(col_val)]
    col_val = sorted(col_val)
    
    n = len(col_val)
    m = round(1 + 3.233 * math.log(n, 10))
    borders = equifreq(col_val, m)
    df[col_name] = pd.cut(df[col_name], bins = borders)
    
apply_binning(so_tang)
apply_binning(so_phong_ngu)
apply_binning(dien_tich)


**Từ chiều dài, chiều rộng, tạo thêm cột dáng nhà biểu diễn hình dán ngôi nhà (so sánh chiều dài và chiều rộng).**

In [9]:
def dang_nha(df):
    if df["Dài (m)"] > df["Rộng (m)"]:
        return "Dài > Rộng"
    elif df["Dài (m)"] == df["Rộng (m)"]:
        return "Dài = Rộng"
    else: return "Dài < Rộng"
    
df['Dáng nhà'] = df.apply(dang_nha, axis = 1)        

**Loại bỏ outliers bằng IOR**

In [10]:
def remove_outliers(df, column_name):
    # Xác định giá trị phân vị thứ 25 và thứ 75
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    
    # Tính khoảng biểu diễn phạm vi bình thường (IQR)
    iqr = q3 - q1
    
    # Xác định giá trị tối thiểu và tối đa không phải là outliers
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Loại bỏ outliers
    filtered_data = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
    return filtered_data

In [11]:
df = remove_outliers(df, 'Giá/m2 (triệu)')

In [12]:
df

,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích (m2),Dài (m),Rộng (m),Giá/m2 (triệu),Dáng nhà
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,"(0.0, 4.0]","(4, 6]","(42, 64]",NaN,NaN,86.96,Dài < Rộng
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",Không có,NaN,"(2, 4]","(31, 42]",NaN,NaN,116.22,Dài < Rộng
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,"(0.0, 4.0]","(2, 4]","(31, 42]",10.00,4.0,65.00,Dài > Rộng
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,"(4, 6]","(42, 64]",12.75,4.0,100.00,Dài > Rộng
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",Không có,NaN,"(2, 4]","(31, 42]",9.00,4.0,86.11,Dài > Rộng
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82489,2019-09-19,"Đường Xuân Thủy, Phường Dịch Vọng Hậu, Quận Cầ...",Quận Cầu Giấy,Phường Dịch Vọng Hậu,"Nhà ngõ, hẻm",Đã có sổ,NaN,"(2, 4]","(31, 42]",NaN,NaN,77.50,Dài < Rộng
82490,2019-08-26,"Đường Lê Đức Thọ, Phường Mỹ Đình 1, Quận Nam T...",Quận Nam Từ Liêm,Phường Mỹ Đình 1,Nhà phố liền kề,Đã có sổ,NaN,"(2, 4]","(31, 42]",NaN,NaN,76.32,Dài < Rộng
82491,2019-08-23,"Đường Hồ Tùng Mậu, Phường Phúc Diễn, Quận Bắc ...",Quận Bắc Từ Liêm,Phường Phúc Diễn,Nhà phố liền kề,Không có,NaN,"(2, 4]","(31, 42]",NaN,NaN,81.58,Dài < Rộng
82494,2019-08-05,"Đường Quan Hoa, Phường Quan Hoa, Quận Cầu Giấy...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà ngõ, hẻm",Đã có sổ,NaN,"(2, 4]","(42, 64]",NaN,NaN,101.67,Dài < Rộng


**Lưu thành file dữ liệu mới tên .**

In [13]:
df.to_csv(OUTPUT)